# Split PDF Book into Multiple Text File

In [37]:
import os
from PyPDF2 import PdfReader, PdfWriter

In [38]:
book = PdfReader('Mining Text Data.pdf')

In [121]:
# actual page number
actual_page = 12
chapter_first_page = [1,11,42,76,128,161,221,256,293,318,355,379,409,458,513]
chapter_first_page[0]+actual_page
page = book.pages[chapter_first_page[6]+actual_page]
print(page.extract_text())

Chapter 7
TRANSFER LEARNING FOR TEXT
MINING
Weike Pan
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
weikep@cse.ust.hk
Erheng Zhong
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
ezhong@cse.ust.hk
Qiang Yang
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
qyang@cse.ust.hk
Abstract Over the years, transfer learning has received much attention in machine
learning research and practice. Researchers have found that a major
bottleneck associated with machine learning and text mining is the lack
of high-quality annotated examples to help train a model. In response,
transfer learning oﬀers an attractive solution for this problem. Various
transfer learning methods are designed to extract the useful knowledge
from diﬀerent but related auxiliary domains. In its connection to text
mining, transfer learning has found novel and useful applications. In
this chapter, we will review some most recent

- Chapter 1: 1-11
- Chapter 2: 11-42
- Chapter 3: 42-76
- Chapter 4: 76-128

In [129]:
book_page_index = [x+12 for x in chapter_first_page]
print(book_page_index)

[13, 23, 54, 88, 140, 173, 233, 268, 305, 330, 367, 391, 421, 470, 525]


In [140]:
for i in range(len(book_page_index)-1):
    print(f'starting at page {book_page_index[i]}, ending at page {book_page_index[i+1]}')
    start_page = book_page_index[i]
    end_page = book_page_index[i+1]
    num_pages = end_page - start_page + 1
    content = ''
    for j in range(num_pages):
        print(j+start_page)
        content += book.pages[j+start_page].extract_text()
    with open(f"doc_{i+1}.txt", "w", encoding='utf-8') as file:
        file.write(content)
        file.close()

starting at page 13, ending at page 23
13
14
15
16
17
18
19
20
21
22
23
starting at page 23, ending at page 54
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
starting at page 54, ending at page 88
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
starting at page 88, ending at page 140
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
starting at page 140, ending at page 173
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
starting at page 173, ending at page 233
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220

# Text Similarity (Chapter Similarity)

In [2]:
import gensim
import nltk
from nltk.corpus import PlaintextCorpusReader

In [3]:
corpus = PlaintextCorpusReader('data', '.+\.txt')
fids = corpus.fileids()
print(fids)

['doc_1.txt', 'doc_10.txt', 'doc_11.txt', 'doc_12.txt', 'doc_13.txt', 'doc_14.txt', 'doc_2.txt', 'doc_3.txt', 'doc_4.txt', 'doc_5.txt', 'doc_6.txt', 'doc_7.txt', 'doc_8.txt', 'doc_9.txt']


In [4]:
docs = [corpus.words(f) for f in fids]

In [7]:
print(docs[0])

['Chapter', '1', 'AN', 'INTRODUCTION', 'TO', 'TEXT', ...]


In [8]:
docs_lower = [[w.lower() for w in doc] for doc in docs]

In [13]:
# Use regular expression to keep only alphabetic words.
import re
docs_alpha = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs_lower]
print(docs_alpha[0][0:20])

# Remove stop words.
from nltk.corpus import stopwords
stop_list = stopwords.words('english')            
docs_stop = [[w for w in doc if w not in stop_list] for doc in docs_alpha]
print(docs_stop[0][0:20])

# Perform Porter stemming. Store in variable docs_stem
from nltk.stem.porter import *
stemmer = PorterStemmer()
docs_stem = [[stemmer.stem(w) for w in doc] for doc in docs_stop]

print(docs_stem[0][0:20])

['chapter', 'an', 'introduction', 'to', 'text', 'mining', 'charu', 'c', 'aggarwal', 'ibm', 't', 'j', 'watson', 'research', 'center', 'yorktown', 'heights', 'ny', 'charu', 'us']
['chapter', 'introduction', 'text', 'mining', 'charu', 'c', 'aggarwal', 'ibm', 'j', 'watson', 'research', 'center', 'yorktown', 'heights', 'ny', 'charu', 'us', 'ibm', 'com', 'chengxiang']
['chapter', 'introduct', 'text', 'mine', 'charu', 'c', 'aggarw', 'ibm', 'j', 'watson', 'research', 'center', 'yorktown', 'height', 'ny', 'charu', 'us', 'ibm', 'com', 'chengxiang']


In [14]:
from gensim import corpora
dictionary = corpora.Dictionary(docs_stem) #dict of unique words
print(dictionary)

Dictionary(11947 unique tokens: ['abil', 'abl', 'abstract', 'access', 'account']...)


In [15]:
token_to_id = dictionary.token2id #word and its id
print(type(token_to_id))
print(token_to_id)

#with this dict for the corpus, now can convert any doc into vector

<class 'dict'>
{'abil': 0, 'abl': 1, 'abstract': 2, 'access': 3, 'account': 4, 'accur': 5, 'achiev': 6, 'acm': 7, 'action': 8, 'adapt': 9, 'add': 10, 'address': 11, 'advanc': 12, 'advancesin': 13, 'afor': 14, 'aggarw': 15, 'aggreg': 16, 'al': 17, 'algo': 18, 'algorithm': 19, 'allow': 20, 'alon': 21, 'also': 22, 'alsob': 23, 'alsoplay': 24, 'amajor': 25, 'amount': 26, 'ampl': 27, 'anal': 28, 'analog': 29, 'analysi': 30, 'analyt': 31, 'analyz': 32, 'andgener': 33, 'andheadquarteredin': 34, 'andi': 35, 'andlda': 36, 'andmani': 37, 'andspe': 38, 'andtechniqu': 39, 'andtop': 40, 'anoth': 41, 'answer': 42, 'anumb': 43, 'appli': 44, 'applic': 45, 'applica': 46, 'approach': 47, 'area': 48, 'arediscuss': 49, 'areoften': 50, 'aris': 51, 'art': 52, 'asgoogl': 53, 'aspect': 54, 'assign': 55, 'associ': 56, 'atedwithalotofcontextinformationsuchasauthor': 57, 'attempt': 58, 'attent': 59, 'attract': 60, 'attribut': 61, 'atur': 62, 'autom': 63, 'avail': 64, 'b': 65, 'baeza': 66, 'bag': 67, 'base': 68, 

In [16]:
doc = docs_stem[1]
print(doc)

['chapter', 'translingu', 'mine', 'text', 'data', 'jian', 'yun', 'nie', 'univers', 'montreal', 'montreal', 'quebec', 'canada', 'nie', 'iro', 'umontr', 'ca', 'jianfeng', 'gao', 'microsoft', 'corpor', 'redmond', 'wa', 'usa', 'jfgao', 'microsoft', 'com', 'guihong', 'cao', 'microsoft', 'corpor', 'redmond', 'wa', 'usa', 'gucao', 'microsoft', 'com', 'abstract', 'like', 'full', 'text', 'translat', 'cross', 'languag', 'inform', 'retriev', 'clir', 'task', 'requir', 'form', 'knowledg', 'transfer', 'across', 'languag', 'although', 'robust', 'translat', 'resourc', 'critic', 'construct', 'high', 'qualiti', 'translat', 'tool', 'manual', 'construct', 'resourc', 'limit', 'coverag', 'adapt', 'wide', 'rang', 'ap', 'plicat', 'automat', 'mine', 'translingu', 'knowledg', 'make', 'po', 'sibl', 'complement', 'hand', 'curat', 'resourc', 'chapter', 'describ', 'grow', 'bodi', 'work', 'seek', 'mine', 'translingu', 'knowledg', 'text', 'data', 'particular', 'data', 'found', 'web', 'review', 'number', 'mine', 'stra

In [17]:
vec = dictionary.doc2bow(doc)
print(vec)

[(0, 2), (1, 2), (2, 2), (4, 1), (6, 1), (7, 3), (9, 5), (10, 2), (12, 1), (15, 2), (17, 50), (19, 4), (20, 2), (21, 2), (22, 17), (26, 4), (28, 1), (29, 4), (30, 3), (41, 8), (44, 6), (45, 13), (46, 1), (47, 26), (48, 2), (51, 1), (52, 5), (54, 2), (55, 3), (56, 4), (58, 5), (60, 1), (63, 1), (64, 6), (65, 10), (67, 1), (68, 50), (71, 1), (72, 1), (73, 2), (75, 1), (83, 2), (85, 79), (87, 7), (88, 1), (89, 1), (90, 18), (93, 2), (99, 1), (101, 1), (103, 7), (104, 3), (107, 38), (108, 1), (110, 2), (112, 3), (116, 11), (117, 11), (118, 2), (119, 1), (121, 8), (122, 1), (124, 1), (125, 45), (127, 1), (129, 16), (131, 1), (132, 2), (133, 8), (135, 6), (136, 1), (137, 9), (138, 1), (140, 19), (141, 10), (142, 6), (143, 2), (148, 27), (149, 29), (150, 11), (151, 4), (152, 5), (153, 1), (154, 1), (155, 4), (156, 22), (158, 2), (161, 73), (166, 4), (170, 13), (171, 2), (172, 1), (173, 5), (175, 11), (178, 1), (182, 3), (183, 1), (187, 4), (188, 1), (189, 2), (190, 7), (191, 1), (192, 53), (1